In [1]:
import numpy as np
import argparse
import os
import sys
import torch
import torch.fft as F
from importlib import reload
from torch.nn.functional import relu
import torch.nn as nn
import torch.nn.functional as Func
import torch.optim as optim
import utils
import mnet
from mnet import MNet
from matplotlib import pyplot as plt
from utils import kplot,mask_naiveRand,mask_filter
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

In [2]:
imgs = torch.tensor( np.load('/home/huangz78/data/data_gt.npz')['imgdata'] ).permute(2,0,1)
base = 24
mask_lf,_,_ = mask_naiveRand(imgs.shape[1],fix=base,other=0,roll=True)

yfulls = torch.zeros((imgs.shape[0],2,imgs.shape[1],imgs.shape[2]),dtype=torch.float)
ys     = torch.zeros((imgs.shape[0],2,imgs.shape[1],imgs.shape[2]),dtype=torch.float)
xs     = torch.zeros(imgs.shape)
for ind in range(imgs.shape[0]):
    imgs[ind,:,:] = imgs[ind,:,:]/torch.max(torch.abs(imgs[ind,:,:]))
    y = torch.fft.fftshift(F.fftn(imgs[ind,:,:],dim=(0,1),norm='ortho'))
    ysub = torch.zeros(y.shape,dtype=y.dtype)
    ysub[mask_lf==1,:] = y[mask_lf==1,:]
    xs[ind,:,:] = torch.abs(F.ifftn(torch.fft.ifftshift(ysub),dim=(0,1),norm='ortho')) 
    
    yfulls[ind,0,:,:] = torch.real(y)
    yfulls[ind,1,:,:] = torch.imag(y)
    ys[ind,:,mask_lf==1,:] = yfulls[ind,:,mask_lf==1,:]
      
labels = torch.tensor( np.load('/home/huangz78/data/data_gt_greedymask.npz')['mask'].T ) 
print(ys.shape)
print(xs.shape)
# labels are already rolled

imgNum = imgs.shape[0]
traininds, testinds = train_test_split(np.arange(imgNum),random_state=0,shuffle=True,train_size=round(imgNum*0.8))
test_total  = testinds.size

traindata   = ys[traininds,:,:,:]
valdata     = ys[testinds[0:test_total//2],:,:,:]
testdata    = ys[testinds[test_total//2:],:,:,:]

trainlabels = mask_filter(labels[traininds,:],base=base)
vallabels   = mask_filter(labels[testinds[0:test_total//2],:],base=base)
testlabels  = mask_filter(labels[testinds[test_total//2:],:],base=base)

# traindata   = xs[traininds,:,:]
# valdata     = xs[testinds[0:test_total//2],:,:]
# traindata = traindata.view(traindata.shape[0],1,traindata.shape[1],traindata.shape[2])
# valdata = valdata.view(valdata.shape[0],1,valdata.shape[1],valdata.shape[2])

torch.Size([199, 2, 320, 320])
torch.Size([199, 320, 320])


In [ ]:
# plt.imshow(xs[0,:,:])
# plt.colorbar()
# plt.show()
# kplot(mask_lf)
# kplot(xs[0,:,:],log=False,roll=False)
kplot(torch.abs(ys[0,0,:,:]),log=True,roll=False)

In [3]:
def sigmoid_binarize(M,threshold=0.5):
    sigmoid = nn.Sigmoid()
    mask = sigmoid(M)
    mask_pred = torch.ones_like(mask)
    for ind in range(M.shape[0]):
        mask_pred[ind,mask[ind,:]<=threshold] = 0
    return mask_pred

# mnet test

In [4]:
reload(mnet)
from mnet import MNet
net = MNet(beta=1,in_channels=traindata.shape[1],out_size=trainlabels.shape[1],\
                   imgsize=(traindata.shape[2],traindata.shape[3]),poolk=3)
net.eval()

MNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(256, eps=1e-05, 

In [6]:
dir_checkpoint = '/home/huangz78/checkpoints/'
net = MNet(beta=1,in_channels=traindata.shape[1],out_size=trainlabels.shape[1],\
                   imgsize=(traindata.shape[2],traindata.shape[3]),poolk=3)
dictpath   = '/home/huangz78/checkpoints/mnet.pth'
checkpoint = torch.load(dictpath)
net.load_state_dict(checkpoint['model_state_dict'])
net.eval()
print('mnet is successfully loaded from the file: ' + dictpath )

mnet is successfully loaded from the file: /home/huangz78/checkpoints/mnet.pth


In [9]:
net.eval()
print('testdata shape is: ', testdata.shape)

imgind = 18
testimg = testdata[imgind,:,:,:]
output_1 = net(testimg.view(-1,2,320,320))
binary_1 = sigmoid_binarize(output_1)[0,:]

imgind = 3
testimg = testdata[imgind,:,:,:]
output_2 = net(testimg.view(-1,2,320,320))
binary_2 = sigmoid_binarize(output_2)[0,:]

print(torch.sum(torch.abs(binary_1-binary_2)))
print(output_1 - output_2)
# sigmoid_binarize(output)[0,:]

testdata shape is:  torch.Size([20, 2, 320, 320])
tensor(10.)
tensor([[ 82.4977,  62.1520,  93.7176,  68.0966,  94.3024,  97.3108,  84.8475,
          89.7607,  66.7971,  93.3319,  87.9182,  68.0016,  94.7396,  84.5460,
          74.8958,  84.4100,  27.3483,  75.4741,  86.1479,  70.1038,  89.7006,
          94.9551,  92.7310,  81.2937, 100.6270,  96.2222,  84.2758,  20.6120,
          84.1709,  82.3289,  79.9693,  76.8130,  93.9968,  89.7806,  90.6824,
          76.4415,  89.1296,  81.1975,  94.2939,  82.8559,  86.6993,  92.6577,
          82.2602,  83.8862,  83.2109,  94.9764,  84.9684,  27.0864,  86.0837,
          85.2859,  20.8679,  24.3479,  86.4053,  71.3624,  92.1977,  68.8290,
         103.2846,  79.7121,  95.3768,  77.6370,  88.4875,  85.6561,  79.9100,
          68.3325,  29.2137,  18.2932,  83.2812, 102.5323,  82.4363,  79.2711,
          85.2263,  78.9588,  91.9754,  -4.2725,  20.7160,  -5.1994,  79.1838,
          25.7344,  -3.9354,  74.9997,  20.4797,  89.8759, 110.5640, 

# mnet train

In [4]:
def trainMNet(trainimgs,trainlabels,testimgs,testlabels,\
              epochs=20,batchsize=5,positive_weight=1,\
              lr=0.01,lr_weight_decay=1e-8,opt_momentum=0,\
              lr_s_stepsize=5,lr_s_gamma=0.5,\
              model=None,save_cp=True,threshold=0.5,\
              beta=1,poolk=3,datatype=torch.float,print_every=10):
    '''
    trainimgs    : train data, with dimension (num. of imgs,layer, height, width)
    '''
    
    train_shape  = trainimgs.shape; test_shape = testimgs.shape 
    trainimgs    = torch.tensor(trainimgs,dtype=datatype)
    trainlabels  = torch.tensor(trainlabels,dtype=datatype)
    testimgs     = torch.tensor(testimgs,dtype=datatype)
    testlabels   = torch.tensor(testlabels,dtype=datatype)    
    dir_checkpoint = '/home/huangz78/checkpoints/'
    # input images are assumed to be normalized
    
    if model is None:
        net = MNet(beta=beta,in_channels=train_shape[1],out_size=trainlabels.shape[1],\
                   imgsize=(train_shape[2],train_shape[3]),poolk=poolk)
    else:
        net = model
#     optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=lr_weight_decay, momentum=opt_momentum)
    optimizer = optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=lr_weight_decay, amsgrad=False)
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=lr_s_stepsize,factor=lr_s_gamma)
    pos_weight = torch.ones([trainlabels.shape[1]]) * positive_weight # weight assigned to positive labels 
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    test_criterion = nn.BCELoss()
    
    epoch_loss        = np.full((epochs),np.nan)
    precision_train   = list([]); recall_train = list([])
    precision_history = np.full((epochs),np.nan); recall_history = np.full((epochs),np.nan)
    net.train()
    for epoch in range(epochs):
        batch_init = 0; step_count = 0
        while batch_init < train_shape[0]:
            batch = np.arange(batch_init,min(batch_init+batchsize,train_shape[0]))
            imgbatch = trainimgs[batch,:,:,:] # maybe shuffling?
            batchlabels = trainlabels[batch,:]
            mask_pred   = net(imgbatch)
            train_loss  = criterion(mask_pred,batchlabels)
            batch_init += batchsize; step_count += 1
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            if (step_count%print_every)==0:
                print('[{}/{}][{}] train batch loss {}'.format(epoch+1,epochs,step_count,train_loss.item()))
                precision_train.append(\
                    precision_score(torch.flatten(batchlabels),\
                                    torch.flatten(sigmoid_binarize(mask_pred,threshold=threshold))) ) 
                recall_train.append(\
                    recall_score(torch.flatten(batchlabels),\
                                 torch.flatten(sigmoid_binarize(mask_pred,threshold=threshold))) )
                print('[{}/{}][{}] precision {}, recall {}'.format(epoch+1,epochs,step_count,precision_train[-1],recall_train[-1]))
        with torch.no_grad():
            net.eval()
            mask_test = sigmoid_binarize(net(testimgs),threshold=threshold)
            test_loss = test_criterion(mask_test,testlabels)
            net.train()
#             scheduler.step(test_loss)
            epoch_loss[epoch] = test_loss.item()
            precision_history[epoch] = precision_score(torch.flatten(testlabels),torch.flatten(mask_test))
            recall_history[epoch] = recall_score(torch.flatten(testlabels),torch.flatten(mask_test))
            print('\t [{}/{}] validation loss {} '.format(epoch+1,epochs,test_loss.item()))
            print('\t [{}/{}] precision {} '.format(epoch+1,epochs,precision_history[epoch]))
            print('\t [{}/{}] recall    {} '.format(epoch+1,epochs,recall_history[epoch]))
        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                print('Created checkpoint directory')
            except OSError:
                pass
            torch.save({'model_state_dict': net.state_dict()}, dir_checkpoint + 'mnet.pth')
#                         'optimizer_state_dict': optimizer.state_dict(),
#                         'epoch': epoch,
#                         'threshold':threshold
#                         }, dir_checkpoint + 'mnet.pth')
#                         }, dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
            print(f'\t Checkpoint saved after epoch {epoch + 1}!')
            np.savez(dir_checkpoint+'epoch_loss.npz', loss=epoch_loss,\
                     precision_train=precision_train,recall_train=recall_train,\
                     precision_test=precision_history,recall_test=recall_history)
        
    return net

In [5]:
model = None
# model = net  
# oldshape = traindata[0,:,:,:].shape
# num = 3
# trainpt = traindata[0:num,:,:,:].view(num,oldshape[0],oldshape[1],oldshape[2])
# trainlb = trainlabels[0:num,:].view(num,-1)
# valpt = valdata[0:num,:,:,:].view(num,oldshape[0],oldshape[1],oldshape[2])
# vallb = vallabels[0:num,:].view(num,-1)
mnet  = trainMNet(traindata, trainlabels, valdata, vallabels,model=model, \
              epochs=400, batchsize=5, \
              positive_weight=1,\
              lr=1e-4, lr_weight_decay=0, opt_momentum=0,\
              lr_s_stepsize=5, lr_s_gamma=0.8,\
              save_cp=True)
# mnet  = trainMNet(trainpt, trainlb, valpt, vallb,model=model, \
#               epochs=150, batchsize=5, \
#               positive_weight=1,\
#               lr=5e-4, lr_weight_decay=0, opt_momentum=0,\
#               lr_s_stepsize=2, lr_s_gamma=0.5,\
#               threshold=.5, beta=1, save_cp=True,print_every=1)

/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is

[1/400][10] train batch loss 0.5894767045974731
[1/400][10] precision 0.5548902195608783, recall 0.695
[1/400][20] train batch loss 0.36093318462371826
[1/400][20] precision 0.7972972972972973, recall 0.7375
[1/400][30] train batch loss 0.3093498945236206
[1/400][30] precision 0.8277777777777777, recall 0.745
	 [1/400] test loss 16.199323654174805 
	 [1/400] precision 0.7230340988169798 
	 [1/400] recall    0.649375 
	 Checkpoint saved after epoch 1!
[2/400][10] train batch loss 0.3324435353279114
[2/400][10] precision 0.7943661971830986, recall 0.705
[2/400][20] train batch loss 0.3015788197517395
[2/400][20] precision 0.7972602739726027, recall 0.7275
[2/400][30] train batch loss 0.2812899947166443
[2/400][30] precision 0.808, recall 0.7575
	 [2/400] test loss 16.233108520507812 
	 [2/400] precision 0.7157326130992573 
	 [2/400] recall    0.6625 
	 Checkpoint saved after epoch 2!
[3/400][10] train batch loss 0.3251744210720062
[3/400][10] precision 0.7896103896103897, recall 0.76
[3/

[19/400][10] train batch loss 0.259626567363739
[19/400][10] precision 0.8847352024922118, recall 0.71
[19/400][20] train batch loss 0.2683403491973877
[19/400][20] precision 0.92, recall 0.6325
[19/400][30] train batch loss 0.25307217240333557
[19/400][30] precision 0.8571428571428571, recall 0.72
	 [19/400] test loss 16.722972869873047 
	 [19/400] precision 0.6920654911838791 
	 [19/400] recall    0.686875 
	 Checkpoint saved after epoch 19!
[20/400][10] train batch loss 0.25252798199653625
[20/400][10] precision 0.9129032258064517, recall 0.7075
[20/400][20] train batch loss 0.2722519040107727
[20/400][20] precision 0.9084507042253521, recall 0.645
[20/400][30] train batch loss 0.2598465085029602
[20/400][30] precision 0.8493975903614458, recall 0.705
	 [20/400] test loss 15.996622085571289 
	 [20/400] precision 0.7135382603008502 
	 [20/400] recall    0.681875 
	 Checkpoint saved after epoch 20!
[21/400][10] train batch loss 0.2510424554347992
[21/400][10] precision 0.9245901639344

	 Checkpoint saved after epoch 36!
[37/400][10] train batch loss 0.24432921409606934
[37/400][10] precision 0.9172185430463576, recall 0.6925
[37/400][20] train batch loss 0.26643893122673035
[37/400][20] precision 0.8961937716262975, recall 0.6475
[37/400][30] train batch loss 0.23798024654388428
[37/400][30] precision 0.9368770764119602, recall 0.705
	 [37/400] test loss 15.067567825317383 
	 [37/400] precision 0.8582995951417004 
	 [37/400] recall    0.53 
	 Checkpoint saved after epoch 37!
[38/400][10] train batch loss 0.24376091361045837
[38/400][10] precision 0.9183006535947712, recall 0.7025
[38/400][20] train batch loss 0.2737780809402466
[38/400][20] precision 0.9072164948453608, recall 0.66
[38/400][30] train batch loss 0.2549097537994385
[38/400][30] precision 0.947565543071161, recall 0.6325
	 [38/400] test loss 16.6047306060791 
	 [38/400] precision 0.8493771234428086 
	 [38/400] recall    0.46875 
	 Checkpoint saved after epoch 38!
[39/400][10] train batch loss 0.24620099

KeyboardInterrupt: 

# data preparation

In [ ]:
base = 24
mask = torch.tensor( mask_naiveRand(320,fix=base,other=0,roll=False)[0] ,dtype=torch.float )
data_under = np.zeros((datashape[2],datashape[0],datashape[1]))
for ind in range(data.shape[2]):
    img = data[:,:,ind]
    img = img/np.max(np.abs(img))
    yfull = F.fftn(torch.tensor(img,dtype=torch.float),dim=(0,1),norm='ortho')
    ypart = torch.tensordot(torch.diag(mask).to(torch.cfloat) , yfull,dims=([1],[0]))
    data_under[ind,:,:] = torch.abs(F.ifftn(ypart,dim=(0,1),norm='ortho'))

In [ ]:
from sklearn.model_selection import train_test_split
imgNum = 199
traininds, testinds = train_test_split(np.arange(imgNum),random_state=0,shuffle=True,train_size=round(imgNum*0.8))
test_total = testinds.size
traindata    = data_under[traininds,:,:]
trainlabels  = mask_filter(labels[traininds,:],base=base)
valdata      = data_under[testinds[0:test_total//2],:,:]
vallabels    = mask_filter(labels[testinds[0:test_total//2],:],base=base)

In [ ]:
print(traindata.shape)
print(trainlabels.shape)
print(valdata.shape)
print(vallabels.shape)

In [ ]:
sys.path.insert(0,'/home/huangz78/mri/mnet/')
import mnet
reload(mnet)
from mnet import MNet

#### training

In [ ]:
# mnet = MNet(out_size=trainlabels.shape[1])
# checkpoint = torch.load('/home/huangz78/mri/checkpoints/mnet.pth')
# mnet.load_state_dict(checkpoint['model_state_dict'])
# print('mnet loaded successfully from : ' + '/home/huangz78/mri/checkpoints/mnet.pth' )
# mnet.train()
# # print(mnet)
trainMNet(traindata,trainlabels, valdata,vallabels,
          epochs=60, batchsize=5, \
          lr=1e-4, lr_weight_decay=0,opt_momentum=0,positive_weight=1,\
          lr_s_stepsize=2,lr_s_gamma=0.5,\
          threshold=.5, beta=1,save_cp=True)